In [1]:
!python --version

Python 3.8.10


In [113]:
pip install xgboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.7 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [119]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import json
import pickle

from scipy.stats import mode

from sklearn.impute import KNNImputer  
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, OrdinalEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

import xgboost as xg

import re
import warnings
warnings.filterwarnings("ignore")

## Problem Statement

## Data Gathering

In [3]:
df = pd.read_csv("train_data.csv")
df

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,6cf51492-02a2-423e-b93d-676f05b9ad53,7c202b37-2add-44e8-9aea-d5b119aea935,Loan Refused,12232,Short Term,7280.0,< 1 year,Rent,46643.0,Debt Consolidation,777.39,18.0,10.0,12,0,6762,7946,0.0,0.0
1,552e7ade-4292-4354-9ff9-c48031697d72,e7217b0a-07ac-47dd-b379-577b5a35b7c6,Loan Refused,25014,Long Term,7330.0,10+ years,Home Mortgage,81099.0,Debt Consolidation,892.09,26.7,NaN,14,0,35706,77961,0.0,0.0
2,9b5e32b3-8d76-4801-afc8-d729d5a2e6b9,0a62fc41-16c8-40b5-92ff-9e4b763ce714,Loan Refused,16117,Short Term,7240.0,9 years,Home Mortgage,60438.0,Home Improvements,1244.02,16.7,32.0,11,1,11275,14815,1.0,0.0
3,5419b7c7-ac11-4be2-a8a7-b131fb6d6dbe,30f36c59-5182-4482-8bbb-5b736849ae43,Loan Refused,11716,Short Term,7400.0,3 years,Rent,34171.0,Debt Consolidation,990.94,10.0,NaN,21,0,7009,43533,0.0,0.0
4,1450910f-9495-4fc9-afaf-9bdf4b9821df,70c26012-bba5-42c0-8dcb-75295ada31bb,Loan Refused,9789,Long Term,6860.0,10+ years,Home Mortgage,47003.0,Home Improvements,503.71,16.7,25.0,13,1,16913,19553,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
111102,503f01e2-c3c4-4b10-886e-f15de57b6f1a,27a17610-1d8c-48a7-b0db-2d81a4fa099a,Loan Given,15654,Long Term,686.0,10+ years,Rent,43050.0,Debt Consolidation,889.69,14.1,49.0,18,0,14730,29757,0.0,0.0
111103,8af53bc0-56a4-4801-a960-52a46a90b33f,c124ca71-f42a-47cc-8ec2-236f7e9868a5,Loan Given,15237,Short Term,706.0,10+ years,Home Mortgage,60948.0,Debt Consolidation,700.9,15.9,48.0,11,0,17733,23060,0.0,0.0
111104,96efd327-f965-42ae-8b2b-d2555205ce19,7d959a54-8eeb-4df6-b582-046f889e4cff,Loan Given,10413,Short Term,719.0,5 years,Home Mortgage,66945.0,Debt Consolidation,1210.59,8.7,NaN,11,0,5205,6348,0.0,0.0
111105,ec19166c-70c5-4d49-adb6-fb3fc15024a2,5723fbf2-1ce8-4752-8bee-1bebf339f278,Loan Given,34447,Short Term,626.0,10+ years,Home Mortgage,78736.0,Debt Consolidation,1476.3,30.4,9.0,12,0,32318,37580,0.0,0.0


## Data Validation

In [4]:
df.duplicated().sum()    # drop all duplicates

5869

In [5]:
df.drop_duplicates(inplace = True, ignore_index= True)

In [6]:
df.duplicated().sum()

0

In [7]:
df["Credit Score"] = np.where(df["Credit Score"]>800,df["Credit Score"]/10,df["Credit Score"])
df

,Loan ID,Customer ID,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Purpose,Monthly Debt,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Maximum Open Credit,Bankruptcies,Tax Liens
0,6cf51492-02a2-423e-b93d-676f05b9ad53,7c202b37-2add-44e8-9aea-d5b119aea935,Loan Refused,12232,Short Term,728.0,< 1 year,Rent,46643.0,Debt Consolidation,777.39,18.0,10.0,12,0,6762,7946,0.0,0.0
1,552e7ade-4292-4354-9ff9-c48031697d72,e7217b0a-07ac-47dd-b379-577b5a35b7c6,Loan Refused,25014,Long Term,733.0,10+ years,Home Mortgage,81099.0,Debt Consolidation,892.09,26.7,NaN,14,0,35706,77961,0.0,0.0
2,9b5e32b3-8d76-4801-afc8-d729d5a2e6b9,0a62fc41-16c8-40b5-92ff-9e4b763ce714,Loan Refused,16117,Short Term,724.0,9 years,Home Mortgage,60438.0,Home Improvements,1244.02,16.7,32.0,11,1,11275,14815,1.0,0.0
3,5419b7c7-ac11-4be2-a8a7-b131fb6d6dbe,30f36c59-5182-4482-8bbb-5b736849ae43,Loan Refused,11716,Short Term,740.0,3 years,Rent,34171.0,Debt Consolidation,990.94,10.0,NaN,21,0,7009,43533,0.0,0.0
4,1450910f-9495-4fc9-afaf-9bdf4b9821df,70c26012-bba5-42c0-8dcb-75295ada31bb,Loan Refused,9789,Long Term,686.0,10+ years,Home Mortgage,47003.0,Home Improvements,503.71,16.7,25.0,13,1,16913,19553,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105233,503f01e2-c3c4-4b10-886e-f15de57b6f1a,27a17610-1d8c-48a7-b0db-2d81a4fa099a,Loan Given,15654,Long Term,686.0,10+ years,Rent,43050.0,Debt Consolidation,889.69,14.1,49.0,18,0,14730,29757,0.0,0.0
105234,8af53bc0-56a4-4801-a960-52a46a90b33f,c124ca71-f42a-47cc-8ec2-236f7e9868a5,Loan Given,15237,Short Term,706.0,10+ years,Home Mortgage,60948.0,Debt Consolidation,700.9,15.9,48.0,11,0,17733,23060,0.0,0.0
105235,96efd327-f965-42ae-8b2b-d2555205ce19,7d959a54-8eeb-4df6-b582-046f889e4cff,Loan Given,10413,Short Term,719.0,5 years,Home Mortgage,66945.0,Debt Consolidation,1210.59,8.7,NaN,11,0,5205,6348,0.0,0.0
105236,ec19166c-70c5-4d49-adb6-fb3fc15024a2,5723fbf2-1ce8-4752-8bee-1bebf339f278,Loan Given,34447,Short Term,626.0,10+ years,Home Mortgage,78736.0,Debt Consolidation,1476.3,30.4,9.0,12,0,32318,37580,0.0,0.0


## EDA

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105238 entries, 0 to 105237
Data columns (total 19 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan ID                       105238 non-null  object 
 1   Customer ID                   105238 non-null  object 
 2   Loan Status                   105238 non-null  object 
 3   Current Loan Amount           105238 non-null  int64  
 4   Term                          105238 non-null  object 
 5   Credit Score                  83900 non-null   float64
 6   Years in current job          100788 non-null  object 
 7   Home Ownership                105238 non-null  object 
 8   Annual Income                 83900 non-null   float64
 9   Purpose                       105238 non-null  object 
 10  Monthly Debt                  105238 non-null  object 
 11  Years of Credit History       105238 non-null  float64
 12  Months since last delinquent  49003 non-null

In [9]:
df.isna().sum()

Loan ID                             0
Customer ID                         0
Loan Status                         0
Current Loan Amount                 0
Term                                0
Credit Score                    21338
Years in current job             4450
Home Ownership                      0
Annual Income                   21338
Purpose                             0
Monthly Debt                        0
Years of Credit History             0
Months since last delinquent    56235
Number of Open Accounts             0
Number of Credit Problems           0
Current Credit Balance              0
Maximum Open Credit                 0
Bankruptcies                      218
Tax Liens                          11
dtype: int64

#### Loan ID

In [10]:
df["Loan ID"].nunique()

88910

#### Customer ID

In [11]:
df["Customer ID"].value_counts()

cfd59fd2-454f-45c7-9a29-17f87fc0f768    2
d4efbe22-ef06-4bcc-a739-a559863c7470    2
ac73fc48-646d-464c-bddf-de20a09f9082    2
f8b33432-aecb-4fa6-9f94-f5e10a2fd72e    2
93a661f0-0717-4b9c-8e10-713197e15df7    2
                                       ..
f81d9a27-4f66-4a53-a2e2-fcfdff50cbd6    1
82ec29b4-041a-4b0f-bb75-3a6949e51142    1
85df270b-35a2-4f76-8391-037cb09ace00    1
159a5960-6317-48c5-96a1-0e9b09d019c9    1
5514bfcc-e996-4d9b-80d4-9ea66dfa1ac5    1
Name: Customer ID, Length: 88910, dtype: int64

In [12]:
# we will drop Loan ID and Customer ID

#### Loan Status

In [13]:
df["Loan Status"].value_counts()
# it is our target column do Label Encoding 

Loan Given      80065
Loan Refused    25173
Name: Loan Status, dtype: int64

#### Current Loan Amount

In [14]:
df["Current Loan Amount"].value_counts()  

99999999    12738
10151          30
10141          28
10166          28
9827           28
            ...  
2106            1
19360           1
2304            1
11347           1
24629           1
Name: Current Loan Amount, Length: 22541, dtype: int64

In [15]:
df["Current Loan Amount"].nunique()

22541

#### Term 

In [16]:
df["Term"].value_counts()     
# do incoding or replace

Short Term    76719
Long Term     28519
Name: Term, dtype: int64

#### Credit Score

In [17]:
df["Credit Score"].nunique()      
# fill Missing Values and data cleaning

167

#### Years in current job

In [18]:
df["Years in current job"].value_counts() 
# data cleaning

10+ years    32663
2 years       9633
3 years       8620
< 1 year      8582
5 years       7188
1 year        6791
4 years       6466
6 years       6028
7 years       5841
8 years       4845
9 years       4131
Name: Years in current job, dtype: int64

#### Home Ownership

In [19]:
df["Home Ownership"].value_counts() 
# one hot encoding

Home Mortgage    51026
Rent             44402
Own Home          9601
HaveMortgage       209
Name: Home Ownership, dtype: int64

#### Annual Income

In [20]:
df["Annual Income"].value_counts()

61188.0     23
51025.0     20
60198.0     17
51230.0     17
60000.0     17
            ..
110066.0     1
43268.0      1
46690.0      1
21716.0      1
123454.0     1
Name: Annual Income, Length: 37983, dtype: int64

In [21]:
df["Annual Income"].nunique()

37983

#### Purpose

In [22]:
df["Purpose"].value_counts()
# one hot encoding

Debt Consolidation      82847
other                    6218
Home Improvements        6140
Other                    3392
Business Loan            1634
Buy a Car                1364
Medical Bills            1184
Buy House                 703
Take a Trip               563
major_purchase            394
small_business            300
moving                    156
wedding                   122
Educational Expenses      109
vacation                  102
renewable_energy           10
Name: Purpose, dtype: int64

#### Monthly Debt

In [23]:
df["Monthly Debt"].value_counts()

0          55
0.0        25
560.42      6
837         6
621.92      6
           ..
981.24      1
846.96      1
173.68      1
1273.27     1
668.71      1
Name: Monthly Debt, Length: 90362, dtype: int64

In [24]:
df["Monthly Debt"].nunique()

90362

#### Years of Credit History 

In [25]:
df["Years of Credit History"].value_counts()

16.0    1416
15.0    1368
17.0    1292
16.5    1237
14.0    1215
        ... 
53.8       1
56.1       1
62.5       1
58.0       1
51.3       1
Name: Years of Credit History, Length: 508, dtype: int64

In [26]:
df["Years of Credit History"].nunique()

508

#### Months since last delinquent

In [27]:
df["Months since last delinquent"].value_counts()

13.0     947
12.0     933
15.0     927
8.0      901
9.0      900
        ... 
139.0      1
130.0      1
115.0      1
141.0      1
143.0      1
Name: Months since last delinquent, Length: 116, dtype: int64

In [28]:
df["Months since last delinquent"].nunique()

116

#### Number of Open Accounts

In [29]:
df["Number of Open Accounts"].value_counts()

9     9780
10    9535
8     9306
11    9028
7     8595
12    7833
6     7143
13    6541
14    5465
5     4951
15    4499
16    3693
4     3017
17    3016
18    2321
19    1987
20    1505
3     1415
21    1155
22     883
23     727
24     575
2      448
25     432
26     286
27     272
28     173
29     127
30     116
31      78
32      66
33      54
34      38
36      28
1       26
35      26
37      18
38      12
43      11
40      10
39       9
41       7
45       5
42       5
47       5
48       4
44       3
52       2
56       2
0        2
76       2
55       1
Name: Number of Open Accounts, dtype: int64

In [30]:
df["Number of Open Accounts"].nunique()

52

#### Number of Credit Problems

In [31]:
df["Number of Credit Problems"].value_counts()

0     90729
1     12563
2      1336
3       383
4       131
5        56
6        16
7         8
9         5
8         4
10        3
11        2
12        1
15        1
Name: Number of Credit Problems, dtype: int64

In [32]:
df["Number of Credit Problems"].nunique()

14

#### Current Credit Balance

In [33]:
df["Current Credit Balance"].value_counts()

0        607
3563      18
7253      17
9262      17
5279      17
        ... 
2742       1
44851      1
71223      1
50135      1
44728      1
Name: Current Credit Balance, Length: 33716, dtype: int64

In [34]:
df["Current Credit Balance"].nunique()

33716

#### Maximum Open Credit

In [35]:
df["Maximum Open Credit"].value_counts()

0        450
0        269
17464      9
15847      9
7412       8
        ... 
39476      1
17745      1
26104      1
45043      1
10634      1
Name: Maximum Open Credit, Length: 67309, dtype: int64

In [36]:
df["Maximum Open Credit"].nunique()

67309

#### Bankruptcies

In [37]:
df["Bankruptcies"].value_counts()

0.0    93539
1.0    10916
2.0      428
3.0      101
4.0       27
5.0        7
7.0        1
6.0        1
Name: Bankruptcies, dtype: int64

In [38]:
df["Bankruptcies"].nunique()

8

#### Tax Liens

In [39]:
df["Tax Liens"].value_counts()

0.0     103233
1.0       1385
2.0        382
3.0        115
4.0         65
5.0         19
6.0         10
7.0          7
9.0          5
8.0          2
11.0         2
15.0         1
10.0         1
Name: Tax Liens, dtype: int64

In [40]:
df["Tax Liens"].nunique()

13

## Feature Engineering

In [41]:
# Aim is to handle missing value, encoding and data modification

In [42]:
df.describe()

,Current Loan Amount,Credit Score,Annual Income,Years of Credit History,Months since last delinquent,Number of Open Accounts,Number of Credit Problems,Current Credit Balance,Bankruptcies,Tax Liens
count,1.052380e+05,83900.000000,8.390000e+04,105238.000000,49003.000000,105238.000000,105238.000000,1.052380e+05,105020.000000,105227.000000
mean,1.211641e+07,717.399905,7.241759e+04,18.229131,34.918883,11.122551,0.166252,1.545967e+04,0.116464,0.029137
std,3.261291e+07,28.219096,5.650503e+04,7.026107,21.959398,5.002925,0.482317,1.957319e+04,0.349179,0.261156
min,4.910000e+02,585.000000,4.033000e+03,3.600000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000
25%,8.174000e+03,704.000000,4.465800e+04,13.500000,16.000000,8.000000,0.000000,5.910250e+03,0.000000,0.000000
50%,1.421950e+04,724.000000,6.161400e+04,17.000000,32.000000,10.000000,0.000000,1.102550e+04,0.000000,0.000000
75%,2.400100e+04,739.000000,8.683400e+04,21.700000,51.000000,14.000000,0.000000,1.933150e+04,0.000000,0.000000
max,1.000000e+08,751.000000,8.713547e+06,70.500000,176.000000,76.000000,15.000000,1.730472e+06,7.000000,15.000000


In [43]:
df.drop(["Loan ID","Customer ID"], axis = 1, inplace = True)

## Encoding

### Loan Status

In [44]:
le = LabelEncoder()
df["Loan Status"] = le.fit_transform(df["Loan Status"])

### Term

In [45]:
df["Term"].replace({"Short Term":0,"Long Term":1}, inplace = True)

### Home Ownership

In [46]:
df["Home Ownership"].replace({"Home Mortgage":0,
                    "Rent":1, "Own Home":2, 
                    "HaveMortgage":3}, inplace = True)

### Purpose

In [47]:
ohe = OneHotEncoder()
ar1= ohe.fit_transform(df[["Purpose"]]).toarray()
df1= pd.DataFrame(ar1,columns=df["Purpose"].unique())
df1

,Debt Consolidation,Home Improvements,other,Business Loan,small_business,Other,moving,Buy a Car,Medical Bills,Buy House,Take a Trip,vacation,major_purchase,Educational Expenses,wedding,renewable_energy
0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105233,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105234,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105235,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105236,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
df2 = df.join(df1)

In [49]:
df2.drop(["Purpose"], axis = 1, inplace = True)
df2

,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,...,moving,Buy a Car,Medical Bills,Buy House,Take a Trip,vacation,major_purchase,Educational Expenses,wedding,renewable_energy
0,1,12232,0,728.0,< 1 year,1,46643.0,777.39,18.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,25014,1,733.0,10+ years,0,81099.0,892.09,26.7,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,16117,0,724.0,9 years,0,60438.0,1244.02,16.7,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,11716,0,740.0,3 years,1,34171.0,990.94,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,9789,1,686.0,10+ years,0,47003.0,503.71,16.7,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105233,0,15654,1,686.0,10+ years,1,43050.0,889.69,14.1,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105234,0,15237,0,706.0,10+ years,0,60948.0,700.9,15.9,48.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105235,0,10413,0,719.0,5 years,0,66945.0,1210.59,8.7,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105236,0,34447,0,626.0,10+ years,0,78736.0,1476.3,30.4,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105238 entries, 0 to 105237
Data columns (total 32 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan Status                   105238 non-null  int64  
 1   Current Loan Amount           105238 non-null  int64  
 2   Term                          105238 non-null  int64  
 3   Credit Score                  83900 non-null   float64
 4   Years in current job          100788 non-null  object 
 5   Home Ownership                105238 non-null  int64  
 6   Annual Income                 83900 non-null   float64
 7   Monthly Debt                  105238 non-null  object 
 8   Years of Credit History       105238 non-null  float64
 9   Months since last delinquent  49003 non-null   float64
 10  Number of Open Accounts       105238 non-null  int64  
 11  Number of Credit Problems     105238 non-null  int64  
 12  Current Credit Balance        105238 non-nul

### Years in current job

In [51]:
df2["Years in current job"].mode()

0    10+ years
Name: Years in current job, dtype: object

### Bankruptcies and Tax Liens

In [52]:
mode1=df2["Bankruptcies"].mode()[0]
mode2 = df2["Tax Liens"].mode()[0]

df2["Bankruptcies"].fillna(mode1,inplace= True)
df2["Tax Liens"].fillna(mode2,inplace= True)

In [53]:
df2

,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,...,moving,Buy a Car,Medical Bills,Buy House,Take a Trip,vacation,major_purchase,Educational Expenses,wedding,renewable_energy
0,1,12232,0,728.0,< 1 year,1,46643.0,777.39,18.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,25014,1,733.0,10+ years,0,81099.0,892.09,26.7,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,16117,0,724.0,9 years,0,60438.0,1244.02,16.7,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,11716,0,740.0,3 years,1,34171.0,990.94,10.0,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,9789,1,686.0,10+ years,0,47003.0,503.71,16.7,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105233,0,15654,1,686.0,10+ years,1,43050.0,889.69,14.1,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105234,0,15237,0,706.0,10+ years,0,60948.0,700.9,15.9,48.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105235,0,10413,0,719.0,5 years,0,66945.0,1210.59,8.7,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105236,0,34447,0,626.0,10+ years,0,78736.0,1476.3,30.4,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
df2.isna().sum()

Loan Status                         0
Current Loan Amount                 0
Term                                0
Credit Score                    21338
Years in current job             4450
Home Ownership                      0
Annual Income                   21338
Monthly Debt                        0
Years of Credit History             0
Months since last delinquent    56235
Number of Open Accounts             0
Number of Credit Problems           0
Current Credit Balance              0
Maximum Open Credit                 0
Bankruptcies                        0
Tax Liens                           0
Debt Consolidation                  0
Home Improvements                   0
other                               0
Business Loan                       0
small_business                      0
Other                               0
moving                              0
Buy a Car                           0
Medical Bills                       0
Buy House                           0
Take a Trip 

### Years in current job

In [57]:
def get_digit(s):
    if type(s) == str:
        l = re.findall("\d+",s)[0]
        return int(l)
    else:
        return s


In [58]:
df2["Years in current job"] = df2["Years in current job"].apply(get_digit)

In [60]:
df2.isna().sum()

Loan Status                         0
Current Loan Amount                 0
Term                                0
Credit Score                    21338
Years in current job             4450
Home Ownership                      0
Annual Income                   21338
Monthly Debt                        0
Years of Credit History             0
Months since last delinquent    56235
Number of Open Accounts             0
Number of Credit Problems           0
Current Credit Balance              0
Maximum Open Credit                 0
Bankruptcies                        0
Tax Liens                           0
Debt Consolidation                  0
Home Improvements                   0
other                               0
Business Loan                       0
small_business                      0
Other                               0
moving                              0
Buy a Car                           0
Medical Bills                       0
Buy House                           0
Take a Trip 

In [61]:
df2["Years in current job"].value_counts()

10.0    32663
1.0     15373
2.0      9633
3.0      8620
5.0      7188
4.0      6466
6.0      6028
7.0      5841
8.0      4845
9.0      4131
Name: Years in current job, dtype: int64

### Monthly Debt 

In [65]:
def remove_dollar(s):
    if type(s) == str:
        a = s.replace("$","")
        return(float(a))

In [66]:
df2["Monthly Debt"] = df2["Monthly Debt"].apply(remove_dollar)

### Maximum Open Credit

In [ ]:
def change_datatype(s):
    if type(s) == str:
        return(float(s))

In [68]:
#df2["Maximum Open Credit"].astype(float)  #  we got error coz we found #VALUE! in column

np.where(df["Maximum Open Credit"] == "#VALUE!")

(array([ 4930, 37357]),)

In [72]:
df2["Maximum Open Credit"].replace("#VALUE!", np.nan, inplace = True)

In [76]:
df2["Maximum Open Credit"] = df2["Maximum Open Credit"].astype(float)

In [77]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105238 entries, 0 to 105237
Data columns (total 32 columns):
 #   Column                        Non-Null Count   Dtype  
---  ------                        --------------   -----  
 0   Loan Status                   105238 non-null  int64  
 1   Current Loan Amount           105238 non-null  int64  
 2   Term                          105238 non-null  int64  
 3   Credit Score                  83900 non-null   float64
 4   Years in current job          100788 non-null  float64
 5   Home Ownership                105238 non-null  int64  
 6   Annual Income                 83900 non-null   float64
 7   Monthly Debt                  65536 non-null   float64
 8   Years of Credit History       105238 non-null  float64
 9   Months since last delinquent  49003 non-null   float64
 10  Number of Open Accounts       105238 non-null  int64  
 11  Number of Credit Problems     105238 non-null  int64  
 12  Current Credit Balance        105238 non-nul

In [79]:
knn_imputer = KNNImputer(n_neighbors= 5)
ar2 = knn_imputer.fit_transform(df2)
df3 = pd.DataFrame(ar2,columns=df2.columns)
df3

,Loan Status,Current Loan Amount,Term,Credit Score,Years in current job,Home Ownership,Annual Income,Monthly Debt,Years of Credit History,Months since last delinquent,...,moving,Buy a Car,Medical Bills,Buy House,Take a Trip,vacation,major_purchase,Educational Expenses,wedding,renewable_energy
0,1.0,12232.0,0.0,728.0,1.0,1.0,46643.0,777.390,18.0,10.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,25014.0,1.0,733.0,10.0,0.0,81099.0,892.090,26.7,33.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,16117.0,0.0,724.0,9.0,0.0,60438.0,1244.020,16.7,32.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,11716.0,0.0,740.0,3.0,1.0,34171.0,990.940,10.0,48.4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,9789.0,1.0,686.0,10.0,0.0,47003.0,503.710,16.7,25.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105233,0.0,15654.0,1.0,686.0,10.0,1.0,43050.0,1199.386,14.1,49.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105234,0.0,15237.0,0.0,706.0,10.0,0.0,60948.0,1226.812,15.9,48.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105235,0.0,10413.0,0.0,719.0,5.0,0.0,66945.0,682.404,8.7,22.8,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
105236,0.0,34447.0,0.0,626.0,10.0,0.0,78736.0,1950.254,30.4,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [80]:
df3.isna().sum()

Loan Status                     0
Current Loan Amount             0
Term                            0
Credit Score                    0
Years in current job            0
Home Ownership                  0
Annual Income                   0
Monthly Debt                    0
Years of Credit History         0
Months since last delinquent    0
Number of Open Accounts         0
Number of Credit Problems       0
Current Credit Balance          0
Maximum Open Credit             0
Bankruptcies                    0
Tax Liens                       0
Debt Consolidation              0
Home Improvements               0
other                           0
Business Loan                   0
small_business                  0
Other                           0
moving                          0
Buy a Car                       0
Medical Bills                   0
Buy House                       0
Take a Trip                     0
vacation                        0
major_purchase                  0
Educational Ex

In [87]:
df3.to_csv("clean_data.csv")

## Model Training

In [81]:
x = df3.drop("Loan Status", axis = 1)
y = df3["Loan Status"]

In [82]:
x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=0.25,random_state=45,stratify=y)
log_model = LogisticRegression()
log_model.fit(x_train,y_train)

LogisticRegression()

## Evaluation Matrix

### Logistic Regression

In [89]:
# Training

y_pred_train = log_model.predict(x_train)

accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_train,y_pred_train)
print("classification report\n",cl_report)
print("-"*40)


accuracy =  0.7607439691871072
----------------------------------------
confusion matrix
 [[60044     4]
 [18880     0]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.76      1.00      0.86     60048
         1.0       0.00      0.00      0.00     18880

    accuracy                           0.76     78928
   macro avg       0.38      0.50      0.43     78928
weighted avg       0.58      0.76      0.66     78928

----------------------------------------


In [84]:
# Testing

y_pred_test = log_model.predict(x_test)

accuracy = accuracy_score(y_test,y_pred_test)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_test,y_pred_test)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_test,y_pred_test)
print("classification report\n",cl_report)
print("-"*40)


accuracy =  0.7607753705815279
----------------------------------------
confusion matrix
 [[20016     1]
 [ 6293     0]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.76      1.00      0.86     20017
         1.0       0.00      0.00      0.00      6293

    accuracy                           0.76     26310
   macro avg       0.38      0.50      0.43     26310
weighted avg       0.58      0.76      0.66     26310

----------------------------------------


## KNN

In [93]:
knn_clf = KNeighborsClassifier()
knn_clf.fit(x_train,y_train)

KNeighborsClassifier()

In [95]:
# Training

y_pred_train = knn_clf.predict(x_train)

accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_train,y_pred_train)
print("classification report\n",cl_report)
print("-"*40)


accuracy =  0.7977903912426515
----------------------------------------
confusion matrix
 [[56726  3322]
 [12638  6242]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.82      0.94      0.88     60048
         1.0       0.65      0.33      0.44     18880

    accuracy                           0.80     78928
   macro avg       0.74      0.64      0.66     78928
weighted avg       0.78      0.80      0.77     78928

----------------------------------------


In [96]:
# Testing

y_pred_test = knn_clf.predict(x_test)

accuracy = accuracy_score(y_test,y_pred_test)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_test,y_pred_test)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_test,y_pred_test)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.7204484986697073
----------------------------------------
confusion matrix
 [[17896  2121]
 [ 5234  1059]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.77      0.89      0.83     20017
         1.0       0.33      0.17      0.22      6293

    accuracy                           0.72     26310
   macro avg       0.55      0.53      0.53     26310
weighted avg       0.67      0.72      0.68     26310

----------------------------------------


## KNN Hyper

### GridSearchCV

In [112]:
knn_class_grid = KNeighborsClassifier()
hyperparameter = {"n_neighbors":np.arange(2,20),
                 "p":[1,2]}
gscv_knn_class = GridSearchCV(knn_class_grid,hyperparameter,cv=5)
gscv_knn_class.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'n_neighbors': array([ 2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19]),
                         'p': [1, 2]})

In [115]:
gscv_knn_class.best_estimator_

KNeighborsClassifier(n_neighbors=18, p=1)

In [116]:
gscv_knn_class = KNeighborsClassifier(n_neighbors=18, p=1)
gscv_knn_class.fit(x_train,y_train)

KNeighborsClassifier(n_neighbors=18, p=1)

In [117]:
# Training

y_pred_train = gscv_knn_class.predict(x_train)

accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_train,y_pred_train)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.7652163997567403
----------------------------------------
confusion matrix
 [[59310   738]
 [17793  1087]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.77      0.99      0.86     60048
         1.0       0.60      0.06      0.10     18880

    accuracy                           0.77     78928
   macro avg       0.68      0.52      0.48     78928
weighted avg       0.73      0.77      0.68     78928

----------------------------------------


In [118]:
# Testing

y_pred_test = gscv_knn_class.predict(x_test)

accuracy = accuracy_score(y_test,y_pred_test)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_test,y_pred_test)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_test,y_pred_test)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.7550361079437476
----------------------------------------
confusion matrix
 [[19645   372]
 [ 6073   220]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.76      0.98      0.86     20017
         1.0       0.37      0.03      0.06      6293

    accuracy                           0.76     26310
   macro avg       0.57      0.51      0.46     26310
weighted avg       0.67      0.76      0.67     26310

----------------------------------------


## Decision Tree

In [98]:
dt_clf = DecisionTreeClassifier()
dt_clf.fit(x_train,y_train)

DecisionTreeClassifier()

In [99]:
# Training

y_pred_train = dt_clf.predict(x_train)

accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_train,y_pred_train)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  1.0
----------------------------------------
confusion matrix
 [[60048     0]
 [    0 18880]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     60048
         1.0       1.00      1.00      1.00     18880

    accuracy                           1.00     78928
   macro avg       1.00      1.00      1.00     78928
weighted avg       1.00      1.00      1.00     78928

----------------------------------------


In [100]:
# Testing

y_pred_test = dt_clf.predict(x_test)

accuracy = accuracy_score(y_test,y_pred_test)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_test,y_pred_test)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_test,y_pred_test)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.683846446218168
----------------------------------------
confusion matrix
 [[15870  4147]
 [ 4171  2122]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.79      0.79      0.79     20017
         1.0       0.34      0.34      0.34      6293

    accuracy                           0.68     26310
   macro avg       0.57      0.57      0.57     26310
weighted avg       0.68      0.68      0.68     26310

----------------------------------------


## Hyper para Decision tree

### GridSearchCV

In [107]:
dt_clf = DecisionTreeClassifier()
hyperparamter = {"criterion": ["Entropy","gini"],
               "max_depth":np.arange(2,7),
               "min_samples_split":np.arange(2,5),
                "min_samples_leaf": np.arange(2,4),
                "random_state":[41,42,45,46]}
gscv = GridSearchCV(dt_clf,hyperparamter,cv=5)
gscv.fit(x_train,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'criterion': ['Entropy', 'gini'],
                         'max_depth': array([2, 3, 4, 5, 6]),
                         'min_samples_leaf': array([2, 3]),
                         'min_samples_split': array([2, 3, 4]),
                         'random_state': [41, 42, 45, 46]})

In [108]:
gscv.best_estimator_

DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, random_state=41)

In [109]:
gscv_model = DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, random_state=41)
gscv_model.fit(x_train,y_train)

DecisionTreeClassifier(max_depth=5, min_samples_leaf=2, random_state=41)

In [110]:
# Training

y_pred_train = gscv_model.predict(x_train)

accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_train,y_pred_train)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.7615548347861342
----------------------------------------
confusion matrix
 [[59904   144]
 [18676   204]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.76      1.00      0.86     60048
         1.0       0.59      0.01      0.02     18880

    accuracy                           0.76     78928
   macro avg       0.67      0.50      0.44     78928
weighted avg       0.72      0.76      0.66     78928

----------------------------------------


In [111]:
# Testing

y_pred_test = gscv_model.predict(x_test)

accuracy = accuracy_score(y_test,y_pred_test)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_test,y_pred_test)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_test,y_pred_test)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.7612314709236032
----------------------------------------
confusion matrix
 [[19957    60]
 [ 6222    71]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.76      1.00      0.86     20017
         1.0       0.54      0.01      0.02      6293

    accuracy                           0.76     26310
   macro avg       0.65      0.50      0.44     26310
weighted avg       0.71      0.76      0.66     26310

----------------------------------------


## RandomForest

In [101]:
rf_model = RandomForestClassifier()
rf_model.fit(x_train,y_train)

RandomForestClassifier()

In [103]:
# Training

y_pred_train = rf_model.predict(x_train)

accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_train,y_pred_train)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  1.0
----------------------------------------
confusion matrix
 [[60048     0]
 [    0 18880]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       1.00      1.00      1.00     60048
         1.0       1.00      1.00      1.00     18880

    accuracy                           1.00     78928
   macro avg       1.00      1.00      1.00     78928
weighted avg       1.00      1.00      1.00     78928

----------------------------------------


In [104]:
# Testing

y_pred_test = rf_model.predict(x_test)

accuracy = accuracy_score(y_test,y_pred_test)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_test,y_pred_test)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_test,y_pred_test)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.7654503990877993
----------------------------------------
confusion matrix
 [[19785   232]
 [ 5939   354]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.77      0.99      0.87     20017
         1.0       0.60      0.06      0.10      6293

    accuracy                           0.77     26310
   macro avg       0.69      0.52      0.48     26310
weighted avg       0.73      0.77      0.68     26310

----------------------------------------


## AdaBoos

In [102]:
adb_model = AdaBoostClassifier()
adb_model.fit(x_train,y_train)

AdaBoostClassifier()

In [106]:
# Training

y_pred_train = adb_model.predict(x_train)

accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_train,y_pred_train)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.7620109466855869
----------------------------------------
confusion matrix
 [[59544   504]
 [18280   600]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.77      0.99      0.86     60048
         1.0       0.54      0.03      0.06     18880

    accuracy                           0.76     78928
   macro avg       0.65      0.51      0.46     78928
weighted avg       0.71      0.76      0.67     78928

----------------------------------------


In [105]:
# Testing

y_pred_test = adb_model.predict(x_test)

accuracy = accuracy_score(y_test,y_pred_test)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_test,y_pred_test)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_test,y_pred_test)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.762789813759027
----------------------------------------
confusion matrix
 [[19852   165]
 [ 6076   217]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.77      0.99      0.86     20017
         1.0       0.57      0.03      0.07      6293

    accuracy                           0.76     26310
   macro avg       0.67      0.51      0.46     26310
weighted avg       0.72      0.76      0.67     26310

----------------------------------------


## xgboost

In [120]:
xg_model = xg.XGBClassifier()
xg_model.fit(x_train,y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_threshold=64, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [121]:
# Training

y_pred_train = xg_model.predict(x_train)

accuracy = accuracy_score(y_train,y_pred_train)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_train,y_pred_train)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_train,y_pred_train)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.8003876951145348
----------------------------------------
confusion matrix
 [[59364   684]
 [15071  3809]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.80      0.99      0.88     60048
         1.0       0.85      0.20      0.33     18880

    accuracy                           0.80     78928
   macro avg       0.82      0.60      0.60     78928
weighted avg       0.81      0.80      0.75     78928

----------------------------------------


In [122]:
# Testing

y_pred_test = xg_model.predict(x_test)

accuracy = accuracy_score(y_test,y_pred_test)
print("accuracy = ",accuracy)
print("-"*40)

cnf = confusion_matrix(y_test,y_pred_test)
print("confusion matrix\n",cnf)
print("-"*40)

cl_report = classification_report(y_test,y_pred_test)
print("classification report\n",cl_report)
print("-"*40)

accuracy =  0.762979855568225
----------------------------------------
confusion matrix
 [[19410   607]
 [ 5629   664]]
----------------------------------------
classification report
               precision    recall  f1-score   support

         0.0       0.78      0.97      0.86     20017
         1.0       0.52      0.11      0.18      6293

    accuracy                           0.76     26310
   macro avg       0.65      0.54      0.52     26310
weighted avg       0.71      0.76      0.70     26310

----------------------------------------


## Model Selection

### Create Pickle file

In [123]:
with open("model.pkl","wb") as f:
    pickle.dump(log_model,f)

### Create Json file

In [124]:
project_data = {"Term":{"Short Term":0,"Long Term":1},
                "Home Ownership":{"Home Mortgage":0,"Rent":1,
                "Own Home":2,"HaveMortgage":3},
               "columns":list(x.columns)}

In [125]:
with open("project_data.json","w") as f:
    json.dump(project_data,f)